In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import plotly.graph_objs as go
from datetime import datetime, date, timedelta
import pytz


In [3]:
# Interval required 5 minutes
data = yf.download(tickers='^spx', period='1d', interval='1m')

# declare figure
fig = go.Figure()

# Candlestick
fig.add_trace(go.Candlestick(x=data.index,
                             open=data['Open'],
                             high=data['High'],
                             low=data['Low'],
                             close=data['Close'], name='market data'))

# Add titles
fig.update_layout(
    title='live share price evolution',
    yaxis_title='price')

# Show
fig.show()


[*********************100%***********************]  1 of 1 completed


In [4]:
# data.head()
# for i, row in data.iterrows():
#     print(i.time())

In [5]:
LIVE = True

if LIVE:
    today = date.today()
else:
    today = datetime(2023,6,8)

def getToday(today):
    year = today.year
    month = today.month
    day = today.day

    return year, month, day

def getYesterday(today):
    yesterday = today - timedelta(days=1)

    year = yesterday.year
    month = yesterday.month
    day = yesterday.day

    return year, month, day

def getTomorrow(today):
    tomorrow = today + timedelta(days=1)

    year = tomorrow.year
    month = tomorrow.month
    day = tomorrow.day

    return year, month, day

# get buyside and sellside liquidity
year, month, day = getYesterday(today)
start = f"{year}-{month}-{day}"
year, month, day = getToday(today)
end = f"{year}-{month}-{day}"
dataForLiquidity = yf.download(tickers='^spx', start=start, end=end, interval='1d')
pSellsideLiquidity = dataForLiquidity.loc[:, "Low"][0]
pBuysideLiquidity = dataForLiquidity.loc[:, "High"][0]
print(f"sellside liquidity: {pSellsideLiquidity}")
print(f"buyside liquidity: {pBuysideLiquidity}")

# get runs on sellside and buyside
year, month, day = getToday(today)
start = f"{year}-{month}-{day}"
year, month, day = getTomorrow(today)
end = f"{year}-{month}-{day}"
sellsideRuns = []
buysideRuns = []
data = yf.download(tickers='^spx', start=start, end=end, interval='5m')
for i, row in data.iterrows():
    if row["Low"] < pSellsideLiquidity:
        print(f"primary sellside liquidity breached at {i} at {row['Low']}")
        sellsideRuns.append((i, row['Low']))
    if row["High"] > pBuysideLiquidity:
        print(f"primary buyside liquidity breached at {i} at {row['High']}")
        sellsideRuns.append((i, row['High']))

# get swing highs and swing lows
swingHighs = []
swingLows = []

year, month, day = getYesterday(today)
start = f"{year}-{month}-{day}"
year, month, day = getToday(today)
end = f"{year}-{month}-{day}"
dataForSwings = yf.download(tickers='^spx', start=start, end=end, interval='5m')
for i, row in dataForSwings.iloc[:-2].iterrows():
    preTime = i
    currTime = i + timedelta(minutes=5)
    postTime = i + timedelta(minutes=10)
    # check for swing high
    if dataForSwings.loc[currTime, "High"] > dataForSwings.loc[preTime, "High"] and dataForSwings.loc[currTime, "High"] > dataForSwings.loc[postTime, "High"]:
        print(f"swing high at {currTime} at {dataForSwings.loc[currTime, 'High']}")
        swingHighs.append((i, dataForSwings.loc[currTime, 'High']))
    # check for swing low
    if dataForSwings.loc[currTime, "Low"] < dataForSwings.loc[preTime, "Low"] and dataForSwings.loc[currTime, "Low"] < dataForSwings.loc[postTime, "Low"]:
        print(f"swing low at {currTime} at {dataForSwings.loc[currTime, 'Low']}")
        swingLows.append((i, dataForSwings.loc[currTime, 'Low']))

year, month, day = getToday(today)
start = f"{year}-{month}-{day}"
year, month, day = getTomorrow(today)
end = f"{year}-{month}-{day}"
dataForSwings = yf.download(
    tickers='^spx', start=start, end=end, interval='5m')
for i, row in dataForSwings.iloc[:-2].iterrows():
    preTime = i
    currTime = i + timedelta(minutes=5)
    postTime = i + timedelta(minutes=10)
    # check for swing high
    if dataForSwings.loc[currTime, "High"] > dataForSwings.loc[preTime, "High"] and dataForSwings.loc[currTime, "High"] > dataForSwings.loc[postTime, "High"]:
        print(
            f"swing high at {currTime} at {dataForSwings.loc[currTime, 'High']}")
        swingHighs.append((i, dataForSwings.loc[currTime, 'High']))
    # check for swing low
    if dataForSwings.loc[currTime, "Low"] < dataForSwings.loc[preTime, "Low"] and dataForSwings.loc[currTime, "Low"] < dataForSwings.loc[postTime, "Low"]:
        print(
            f"swing low at {currTime} at {dataForSwings.loc[currTime, 'Low']}")
        swingLows.append((i, dataForSwings.loc[currTime, 'Low']))


SEARCH_GREEN_FVG = False
SEARCH_RED_FVG = False

if sellsideRuns and swingHighs:
    SEARCH_GREEN_FVG = True
if buysideRuns and swingLows:
    SEARCH_RED_FVG = True


utc = pytz.UTC

year, month, day = getToday(today)

tp1_open = datetime(year, month, day, 10, 0, 0).replace(tzinfo=utc)
tp1_close = datetime(year, month, day, 10, 0, 0).replace(tzinfo=utc)

tp2_open = datetime(year, month, day, 14, 0, 0).replace(tzinfo=utc)
tp2_close = datetime(year, month, day, 15, 0, 0).replace(tzinfo=utc)

intervals = ["5m"]
for interval in intervals:
    dataForFVG = yf.download(tickers='^spx', start=start, end=end, interval=interval)
    if SEARCH_GREEN_FVG:
        for i, row in dataForSwings.iloc[:-2].iterrows():
            preTime = i
            currTime = i + timedelta(minutes=5)
            postTime = i + timedelta(minutes=10)
            if (postTime > tp1_open and postTime < tp1_close) or (postTime > tp2_open and postTime < tp2_close):
                if dataForFVG.loc[postTime, "Low"] - dataForFVG.loc[preTime, "High"] > 0:
                    print(
                        f"FVG Identified at {postTime+ timedelta(minutes=5)}")
                    print(f"Entry price at {dataForFVG.loc[postTime, 'Low']}")
                    pSwingThreshold = dataForFVG.loc[postTime, "Low"] + 10

                    takeProfitPrice = float("inf")
                    for potential in swingHighs:
                        if potential[0] < postTime and potential[1] > pSwingThreshold:
                            if potential[1]-pSwingThreshold < takeProfitPrice-pSwingThreshold:
                                takeProfitPrice = potential[1]
                    
                    if takeProfitPrice != float("inf"):
                        print(f"Take profit at {takeProfitPrice}")
                    else:
                        print("No Valid Profit")
 
    if SEARCH_RED_FVG:
        for i, row in dataForSwings.iloc[:-2].iterrows():
            preTime = i
            currTime = i + timedelta(minutes=5)
            postTime = i + timedelta(minutes=10)
            if (postTime > tp1_open and postTime < tp1_close) or (postTime > tp2_open and postTime < tp2_close):
                if dataForFVG.loc[postTime, "High"] - dataForFVG.loc[preTime, "Low"] < 0:
                    print(f"FVG Identified at {postTime}")
                    print(f"Entry price at {dataForFVG.loc[postTime, 'High']}")
                    pSwingThreshold = dataForFVG.loc[postTime, "High"] - 10

                    takeProfitPrice = 0
                    for potential in swingHighs:
                        if potential[0] < postTime and potential[1] < pSwingThreshold:
                            if pSwingThreshold-potential[1] < pSwingThreshold-takeProfitPrice:
                                takeProfitPrice = potential[1]

                    if takeProfitPrice != 0:
                        print(f"Take profit at {takeProfitPrice}")
                    else:
                        print("No Valid Profit")





[*********************100%***********************]  1 of 1 completed
sellside liquidity: 4263.9599609375
buyside liquidity: 4299.18994140625
[*********************100%***********************]  1 of 1 completed
primary sellside liquidity breached at 2023-06-08 09:45:00-04:00 at 4262.47021484375
primary sellside liquidity breached at 2023-06-08 09:50:00-04:00 at 4261.77001953125
primary sellside liquidity breached at 2023-06-08 09:55:00-04:00 at 4261.06982421875
[*********************100%***********************]  1 of 1 completed
swing high at 2023-06-07 09:40:00-04:00 at 4298.64013671875
swing high at 2023-06-07 09:50:00-04:00 at 4299.18994140625
swing high at 2023-06-07 10:00:00-04:00 at 4296.259765625
swing low at 2023-06-07 10:00:00-04:00 at 4287.72998046875
swing high at 2023-06-07 10:10:00-04:00 at 4293.68994140625
swing low at 2023-06-07 10:25:00-04:00 at 4282.5498046875
swing high at 2023-06-07 10:30:00-04:00 at 4288.9599609375
swing low at 2023-06-07 10:50:00-04:00 at 4274.70996